## **Notebook Description**

**Function**

Uses the trained model to make predictions on a new, unseen dataset. 

**Output**

Csv with predictions on unseen dataset. 

## **Import Libraries**

In [1]:
import numpy as np
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.data import Data

## **Input**

In [2]:
# Paste model path
model_iteration = 'model'

In [3]:
# Input Pytorch Object Date
# Copy paste in here
object = 'run_3'

In [4]:
# Set the slicing range for the indices (labeled nodes)
sensor_indices = slice(0, 720) 

In [5]:
#Path to save script
file_path = rf"data\2_prediction_testing\sensor_predictions.csv"

## **Classes**

**Load the Trained Model**

In [6]:
# Note to self, convert this class to a .py script later on

class SimpleGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)  # First GCN layer
        self.conv2 = GCNConv(16, num_classes)        # Second GCN layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return x  

## **Loading & Settings**

In [7]:
num_node_features = 4  
num_classes = 1  

model = SimpleGCN(num_node_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  

In [8]:
# Load the saved state_dict
model.load_state_dict(torch.load(rf'data\1_pytorch_model\{model_iteration}.pth'))

<All keys matched successfully>

In [9]:
# Set the model to evaluation mode
model.eval()

SimpleGCN(
  (conv1): GCNConv(4, 16)
  (conv2): GCNConv(16, 1)
)

In [10]:
# Load your data object for prediction
new_data = torch.load(rf'data\torch_data_object_prediction\{object}.pt')

print('loaded')

loaded


In [11]:
# If 'new_data' is a single Data object, wrap it in a list for DataLoader compatibility
if isinstance(new_data, Data):
    new_data = [new_data]

In [12]:
# Create a DataLoader for your new dataset
data_loader = DataLoader(new_data, batch_size=1, shuffle=False)

## **Predictions**

In [13]:
# Make predictions
predictions = []
for data in data_loader:
    with torch.no_grad():
        output = model(data)
        sensor_output = output[sensor_indices]  # Adjusted slicing
        predictions.append(sensor_output.squeeze(-1).detach().numpy())

In [14]:
predictions

[array([228.92117, 235.70271, 242.48422, 249.26576, 256.0473 , 262.82883,
        269.61035, 276.3919 , 283.17343, 289.955  , 296.7365 , 303.51804,
        310.29956, 317.0811 , 323.86264, 330.64413, 337.4257 , 344.2072 ,
        350.98874, 357.7703 , 364.5518 , 371.33334, 378.1149 , 384.89642,
        391.67795, 398.4595 , 405.24106, 412.02255, 418.8041 , 230.20404,
        237.22775, 244.25148, 251.2752 , 258.29895, 265.32266, 272.3464 ,
        279.37015, 286.39386, 293.41757, 300.4413 , 307.46506, 314.4888 ,
        321.51254, 328.53625, 335.55997, 342.5837 , 349.60742, 356.63116,
        363.65488, 370.67862, 377.70236, 384.72607, 391.7498 , 398.77353,
        405.79727, 412.82098, 419.84476, 233.02731, 240.05106, 247.07477,
        254.09851, 261.12222, 268.14597, 275.16968, 282.19342, 289.21716,
        296.24088, 303.26462, 310.28833, 317.31204, 324.33582, 331.35953,
        338.38327, 345.40698, 352.43073, 359.45444, 366.47818, 373.50192,
        380.52563, 387.54935, 394.5731

In [15]:
# Display the shape of predictions
predictions_array = np.array(predictions)
print("Shape of predictions array:", predictions_array.shape)

Shape of predictions array: (1, 720)


In [16]:
print("Output shape:", output.shape)
print("Sensor output shape:", sensor_output.shape)

Output shape: torch.Size([3825, 1])
Sensor output shape: torch.Size([720, 1])


## **Export to csv**

In [17]:
# Reshape predictions_array to have shape (592, 1)
predictions_array = np.vstack(predictions)
predictions_array = predictions_array.T
print("Shape of sensor predictions array:", predictions_array.shape)

Shape of sensor predictions array: (720, 1)


In [18]:
# Export rounded predictions to CSV in the same folder as the script
np.savetxt(file_path, predictions_array, delimiter=",")
print(f"Saved predictions to: {file_path}")

Saved predictions to: data\2_prediction_testing\sensor_predictions.csv
